In [1]:
import numpy as np
from shapely.geometry import Polygon

def create_grid(poly, grid_size):
    xmin, ymin, xmax, ymax = poly.bounds
    xcoords = np.arange(xmin, xmax+grid_size, grid_size)
    ycoords = np.arange(ymin, ymax+grid_size, grid_size)
    cells = []
    for x in range(len(xcoords)-1):
        for y in range(len(ycoords)-1):
            cell_poly = Polygon([(xcoords[x], ycoords[y]), 
                                 (xcoords[x+1], ycoords[y]), 
                                 (xcoords[x+1], ycoords[y+1]), 
                                 (xcoords[x], ycoords[y+1])])
            if poly.intersects(cell_poly):
                cells.append(cell_poly)
    return cells


In [4]:
import plotly.graph_objects as go
from shapely.geometry import Polygon

# Create a Shapely convex hull
points = [(0, 0), (1, 2), (2, 1), (3, 3), (4, 1), (5, 2), (6, 0)]
poly = Polygon(points)
hull = poly.convex_hull

# Create grid cells for the convex hull
grid_size = 1
cells = create_grid(hull, grid_size)

# Create a Plotly figure and add the convex hull and grid cells as traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=hull.exterior.xy[0].tolist(), y=hull.exterior.xy[1].tolist(), mode='lines', line=dict(color='blue')))
for cell in cells:
    fig.add_trace(go.Scatter(x=cell.exterior.xy[0].tolist(), y=cell.exterior.xy[1].tolist(), mode='lines', line=dict(color='red')))

# Set the layout and show the figure
fig.update_layout(title='Convex Hull with Grid Cells', xaxis=dict(scaleanchor="y", scaleratio=1), yaxis=dict(scaleanchor="x", scaleratio=1))
fig.show()


In [65]:
import rospkg
import pickle
import plotly.graph_objects as go
from shapely.geometry import Polygon
import numpy as np

dirname = rospkg.RosPack().get_path('mrpp_sumo')
graph_name = 'iit_bombay'
hull_path = dirname+'/graph_ml/'+graph_name+'_hull'
with open(hull_path, "rb") as poly_file:
    hull = pickle.load(poly_file)

grid_size = 300
gride_cell_area = grid_size*grid_size
cells = create_grid(hull, grid_size)
fully_contained_cells = []
for cell in cells:
    if cell.within(hull):
        fully_contained_cells.append(cell)

partially_contained_cells = []
for cell in cells:
    intersection = cell.intersection(hull)
    if intersection.area > 0 and not cell.within(hull):
        partially_contained_cells.append(intersection)

# Convert to tuples to make them hashable
partially_contained_cells = [tuple(cell.exterior.coords) for cell in partially_contained_cells]

# Remove any duplicates
partially_contained_cells = list(set(partially_contained_cells))

# Convert back to Polygon objects
partially_contained_cells = [Polygon(cell) for cell in partially_contained_cells]

contained_cells = fully_contained_cells + partially_contained_cells
# Create a Plotly figure and add the convex hull and grid cells as traces
fig = go.Figure()

# for cell in partially_contained_cells:
#     fig.add_trace(go.Scatter(x=cell.exterior.xy[0].tolist(), y=cell.exterior.xy[1].tolist(), mode='lines', line=dict(color='red')))
# for cell in fully_contained_cells:
#     fig.add_trace(go.Scatter(x=cell.exterior.xy[0].tolist(), y=cell.exterior.xy[1].tolist(), mode='lines', line=dict(color='green')))
area_fraction = []
centers = []
for cell in contained_cells:
    area_fraction.append(np.round(cell.area/gride_cell_area,2))
    centers.append(cell.centroid)
    # print(cell.centroid)
    fig.add_trace(go.Scatter(x=cell.exterior.xy[0].tolist(), y=cell.exterior.xy[1].tolist(), mode='lines', line=dict(color='green')))
    fig.add_trace(go.Scatter(x=[cell.centroid.x], y=[cell.centroid.y], mode='markers', line=dict(color='red')))      

fig.add_trace(go.Scatter(x=hull.exterior.xy[0].tolist(), y=hull.exterior.xy[1].tolist(), mode='lines', line=dict(color='blue')))
# Set the layout and show the figure

fig.update_layout(title='Convex Hull with Grid Cells', xaxis=dict(scaleanchor="y", scaleratio=1), yaxis=dict(scaleanchor="x", scaleratio=1))
fig.show()
# print(area_fraction,centers[0])


AttributeError: 'MultiPolygon' object has no attribute 'exterior'

In [14]:
import numpy as np
from shapely.geometry import Polygon

# Define the grid and the convex hull as before

# Get all grid cells that are fully contained within the convex hull

